### Stripped-down version of 03

Hi all, this is just a stripped-down version of notebook 3, which took about 3 minutes to execute. That sucks.
And yes, I know it should be 'bare' . It's a joke, not a typo.

In [ ]:
%run function_definitions.ipynb
from scipy.optimize import curve_fit

In [ ]:
bg_filenames = [
    './PTB_DATA/2016_MAY_31/2016_MAY_31_2_5MeV_NE213_FG_12.mpa',
    './PTB_DATA/2016_JUN_02/2016_JUN_02_2_5MeV_NE213_FG_1.mpa']
bg_livetimes = [
    36001.78,
    40002.56
    ]
ng_filenames = [
    './PTB_DATA/2016_MAY_31/2016_MAY_31_2_5MeV_NE213_FG_7.mpa', # 50 kV 2.5 mA 0°
    './PTB_DATA/2016_JUN_01/2016_JUN_01_2_5MeV_NE213_FG_1.mpa', # 50 kV 2.5 mA 0° (SAME SETTINGS!)
    ]
ng_livetimes = [
    9806.20,
    11950.19,
    ]

# For the combination
fg_livetime = ng_livetimes[0] + ng_livetimes[1]
bg_livetime = sum(bg_livetimes)

In [ ]:
e_scale = pickle.load(open('./08_ascii_files/e_scale.p', 'rb'))
p_scale_to_e_and = [-14.325000000000003, 2.3875]

In [ ]:
# NG data
spec2d_list = []
for ng_filename in ng_filenames:
    line_n = find_line_start(ng_filename,'[CDAT2')
    spec2d_raw = get_spectrum(ng_filename, line_n)
    spec2d = np.reshape(spec2d_raw, (1024,4096))
    spec2d_list.append(spec2d)
#spec2d = sum(spec2d_list)

# NOTE THIS IS A NEW LINE... Only analize the first run, not all combined!
# NOTE modified AGAIN August 2016. spec2d = spec2d_list[0] -> spec2d = spec2d_list[0] + spec2d_list[1]
spec2d = spec2d_list[0] + spec2d_list[1]

In [ ]:
# BG data
spec2d_bg_list = []
for bg_filename in bg_filenames:
    line_n = find_line_start(bg_filename,'[CDAT2')
    spec2d_raw = get_spectrum(bg_filename, line_n)
    spec2d_bg = np.reshape(spec2d_raw, (1024,4096))
    spec2d_bg_list.append(spec2d_bg)
# Sum the background
spec2d_bg = sum(spec2d_bg_list)

In [ ]:
class Cut:
    '''
    Produce a cut function to cut on a percentile list.
    '''
    
    pct = 0.
    pct_list = [True for x in range(4096) ] 
    x_thr = 0.
    y_thr = 0.
    
    def get_percentile(self, spec2d, pct):
        self.pct_list = []
        self.pct = pct
        for x in range(len(spec2d[0])):
            self.pct_list.append(self.get_percentile_pos(spec2d[:,x],pct))
        
    def get_percentile_pos(self, dist, percent):
        tot = sum(dist)
        count = 0
        for i, x in enumerate(dist):
            if count >= tot*percent/100.:
                return i
            count += x
        # We should never arrive here
        print('WARNING! Error in get_percentile_pos')
        return -1
    
    def f_cut(self, x, y):
        '''
        FALSE = CUT
        TRUE = KEEP
        '''
        cut_y = self.pct_list[x-1]
        if y < cut_y:
            return False
        if x < self.x_thr:
            return False
        if y < self.y_thr:
            return False
        else:
            return True
        
    def plot_bounds(self, **kwargs):
        plt.plot(range(len(self.pct_list)), self.pct_list, **kwargs)
        plt.axvline(self.x_thr, **kwargs)
        plt.axhline(self.y_thr, **kwargs)
        
    def plot_percentile(self, **kwargs):
        plt.plot(range(len(self.pct_list)), self.pct_list, **kwargs)

In [ ]:
cut_list = []
# 99 percent looks good, add to this list if you disagree
for percentile in [99.0]:
    cut = Cut()
    cut.get_percentile(spec2d_bg, percentile)
    cut_list.append(cut)

In [ ]:
cut = cut_list[0]
cut.x_thr = 48
print('Threshold is at %.1f keVee' % e_scale[cut.x_thr])
spec2d_cut = cut_hist2d(spec2d, cut.f_cut)
spec2d_bg_cut = cut_hist2d(spec2d_bg, cut.f_cut)

In [ ]:
# Settings equal for all runs
cut = cut_list[0]
cut.x_thr = 48

# Now, for all runs:
# - Take 2d spectrum
# - Apply cuts based on BG percentiles
# - Project on x axis to obtain 1d (energy) spectrum
# - Divide by lifetime or run 
fg_disc   = 1/fg_livetime*np.sum((cut_hist2d(spec2d_list[0], cut.f_cut) + cut_hist2d(spec2d_list[1], 
                                                                                                 cut.f_cut)), axis=0)




# Calculate errors:
# - Assume only Poissonian errors
# WARNING: - uncertainty on cut is NOT taken into account
# WARNING: - zero error for zero count, screws all fits up.
fg_disc_err   = 1/fg_livetime*np.sqrt(np.sum((cut_hist2d(spec2d_list[0], cut.f_cut) + cut_hist2d(spec2d_list[1], 
                                                                                                 cut.f_cut)), axis=0))


# Also for BG, because this is how much of the BG leaks into signal!
bg_disc = 1/bg_livetime*np.sum(cut_hist2d(spec2d_bg, cut.f_cut), axis=0)
bg_disc_err = 1/bg_livetime*np.sqrt(np.sum(cut_hist2d(spec2d_bg, cut.f_cut), axis=0))

# Subtract BG leakage
fg_disc_sub = fg_disc - bg_disc
fg_disc_sub_err = np.sqrt(fg_disc_err**2 + bg_disc_err**2)

In [ ]:
def get_percentile_location(arr, pct):
    '''
    No idea why fucking numpy doesnt work on 1d arrays gvd
    '''
    total = sum(arr)
    current_sum = 0.
    i = 0
    while current_sum < total*pct:
        current_sum += arr[i]
        i += 1
    return i

In [ ]:
def rebin(arr, factor=2, mode='avg'):
    '''
    Rebin an array. Take integer factor. Modes available:
      - avg: average. Default.
      - sum: Sum.
      - quad: adding all in quadrature. Mostly for errors.
      - quad_avg: adding all in quadrature, then taking average. For errors on averages.
    '''
    if mode == 'avg':
        rebin = np.array([1/factor*sum(arr[i:i + factor]) for i in np.arange(0, len(arr), factor)])
    elif mode == 'sum':
        rebin = np.array([sum(arr[i:i + factor]) for i in np.arange(0, len(arr), factor)])
    elif mode == 'quad':
        rebin = np.array([np.sqrt(sum(arr[i:i + factor]**2))
                          for i in np.arange(0, len(arr), factor)])
    elif mode == 'quad_avg':
        rebin = np.array([1/factor*np.sqrt(sum(arr[i:i + factor]**2))
                          for i in np.arange(0, len(arr), factor)])
    else:
        raise SyntaxError('Did not get the rebinning mode, you specified: %s' % mode)
    
    return rebin

In [ ]:
pct = 0.99
cutoffs = []
e_scale_he_bins = []
fg_he = []
bg_he = []

for limit in np.arange(400, 4096, 50):
    dist_bg = np.sum(spec2d_bg[:, limit:limit+50], axis=1)
    current_cutoff = get_percentile_location(dist_bg, pct)
    cutoffs.append(current_cutoff)
    e_scale_he_bins.append(limit+25)
    fg_he.append(np.sum(spec2d[current_cutoff:, limit:limit+50]))
    bg_he.append(np.sum(spec2d_bg[current_cutoff:, limit:limit+50]))
    

e_scale_he = poly_1(np.array(e_scale_he_bins), *p_scale_to_e_and)

# Make arrays to enable numpy magic!
# NOTE that these are still NUMBERS and not rate!
fg_he = np.array(fg_he)
bg_he = np.array(bg_he)

# Now calculate Poissonian errors
fg_he_err = 1/(ng_livetimes[0] + ng_livetimes[1])*np.sqrt(fg_he)
bg_he_err = 1/sum(bg_livetimes)*np.sqrt(bg_he)

fg_he = 1/(ng_livetimes[0] + ng_livetimes[1])*fg_he
bg_he = 1/sum(bg_livetimes)*bg_he

fg_he_sub = fg_he - bg_he
fg_he_sub_err = np.sqrt(bg_he_err**2 + fg_he_err**2)

In [ ]:
# Need for 'nicer histogram'  instead of imshow
# We make event-by-event lists of values that we can histogram, instead of setting bin content.
spec2d_counts_x = []
spec2d_counts_y = []

for i in range(len(spec2d)):
    for j in range(len(spec2d[0])):
        for k in range(spec2d[i,j]):
            spec2d_counts_y.append(i)
            spec2d_counts_x.append(j)
spec2d_counts_x = np.array(spec2d_counts_x)
spec2d_counts_y = np.array(spec2d_counts_y)

# Same for bg...
spec2d_bg_counts_x = []
spec2d_bg_counts_y = []

for i in range(len(spec2d_bg)):
    for j in range(len(spec2d_bg[0])):
        for k in range(spec2d_bg[i,j]):
            spec2d_bg_counts_y.append(i)
            spec2d_bg_counts_x.append(j)
spec2d_bg_counts_x = np.array(spec2d_bg_counts_x)
spec2d_bg_counts_y = np.array(spec2d_bg_counts_y)

In [ ]:
led_region = np.array(e_scale_he_bins)[np.array(cutoffs) > 200]

# For new values first just take the values we found previously
fg_he_proper = fg_he.copy()
fg_he_err_proper = fg_he_err.copy()
bg_he_proper = bg_he.copy()
bg_he_err_proper = bg_he_err.copy()
fg_he_sub_proper = fg_he_sub.copy()
fg_he_sub_err_proper = fg_he_sub_err.copy()

for i, e in enumerate(e_scale_he_bins):
    if e in led_region:
        fg_he_proper[i] = np.sum(spec2d[170:190, e-25:e+25])
        bg_he_proper[i] = np.sum(spec2d_bg[170:190, e-25:e+25])
        # Now calculate Poissonian errors
        fg_he_err_proper[i] = 1/(ng_livetimes[0] + ng_livetimes[1])*np.sqrt(fg_he_proper[i])
        bg_he_err_proper[i] = 1/sum(bg_livetimes)*np.sqrt(bg_he_proper[i])

        fg_he_proper[i] = 1/(ng_livetimes[0] + ng_livetimes[1])*fg_he_proper[i]
        bg_he_proper[i] = 1/sum(bg_livetimes)*bg_he_proper[i]

        fg_he_sub_proper[i] = fg_he_proper[i] - bg_he_proper[i]
        fg_he_sub_err_proper[i] = np.sqrt(bg_he_err_proper[i]**2 + fg_he_err_proper[i]**2)

In [ ]:
from IPython.utils import io
with io.capture_output() as output_14:
    %run '14_a_closer_look_at_acceptance.ipynb'


### Rebinning

In [ ]:
e_scale_r = rebin(e_scale, mode='avg') # r for rebin
fg_disc_sub_r = rebin(fg_disc_sub, mode='avg')
fg_disc_sub_err_r = rebin(fg_disc_sub_err, mode='quad_avg')
# Manual calculation with other method confirms the error bars
ehm = np.sqrt(abs(rebin(fg_disc_sub, mode = 'sum') * fg_livetime)) / fg_livetime / 2


def correct_for_acc(arr, acceptance_array, cutoff = 1e-6):
    assert len(arr) == len(acceptance_array)
    arr_c = []
    for a, acc in zip(arr, acceptance_array):
        if acc < cutoff:
            arr_c.append(0)
        else:
            arr_c.append(a/acc)
    return np.array(arr_c)

fg_disc_sub_r_a = correct_for_acc(fg_disc_sub_r, acc_e_r)
fg_disc_sub_err_r_a = correct_for_acc(fg_disc_sub_err_r, acc_e_r)



In [ ]:
import time
time.clock()